In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)

In [16]:
idx2chr=['h','i','e','l','o']
x_data=[[0,1,0,2,3,3]]#hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3
y_data=[[1,0,2,3,3,4]]#ihello

num_classes=5
input_dim=5
hidden_size=5
batch_size=1
sequence_length=6
learning_rate=.1


In [11]:
tf.reset_default_graph()

X=tf.placeholder(tf.float32, [None, sequence_length, input_dim])
Y=tf.placeholder(tf.int32, [None, sequence_length])

cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state=cell.zero_state(batch_size, tf.float32)
outputs,_states=tf.nn.dynamic_rnn(cell,X,initial_state=initial_state,dtype=tf.float32)

X_for_fc=tf.reshape(outputs,[-1,hidden_size])
outputs=tf.contrib.layers.fully_connected(inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)
outputs=tf.reshape(outputs,[batch_size, sequence_length, num_classes])

weights=tf.ones([batch_size, sequence_length])
sequence_loss=tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss=tf.reduce_mean(sequence_loss)
train=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction=tf.argmax(outputs,axis=2)

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(50):
        l, _ =sess.run([loss,train],feed_dict={X:x_one_hot, Y:y_data})
        result=sess.run(prediction,feed_dict={X:x_one_hot})
        print(i,'loss:',l,',prediction:',result,'true Y: ',y_data)
        
        result_str=[idx2chr[c] for c in np.squeeze(result)]
        print('\tPrediction str: ',''.join(result_str))

0 loss: 1.6165359 ,prediction: [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
1 loss: 1.484621 ,prediction: [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
2 loss: 1.387331 ,prediction: [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
3 loss: 1.2862667 ,prediction: [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
4 loss: 1.1742209 ,prediction: [[3 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llelll
5 loss: 1.0480208 ,prediction: [[3 2 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  leello
6 loss: 0.9133066 ,prediction: [[0 2 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  heello
7 loss: 0.78828454 ,prediction: [[0 2 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  heello
8 loss: 0.6932487 ,prediction: [[0 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  hhello
9 loss: 0.6206468 ,prediction: [[0 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predi